#### Notebook to feature engineer fraction of time isoprene was above 90th percentile

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 15)

In [2]:
PATH_1=os.path.join('..','..','..','..','Weather_Data','CAMS','2015','2015_multi_level_26_variables.nc')
ml_26_variables = xr.open_dataset(PATH_1) ## multilevel 26 variables
ml_26_variables = ml_26_variables.squeeze(dim="model_level") ## remove the model_level dimension
ml_26_variables=ml_26_variables.drop_vars('model_level') ## the dropped dimension becomes a column, so remove that
ml_26_variables['longitude'] = ml_26_variables['longitude']-360 ## necessary because of longitude data in multi-level case
ml_26_variables
isoprene=ml_26_variables['c5h8'] ## get only isoprene variable
isoprene

<xarray.DataArray 'c5h8' (valid_time: 2680, latitude: 34, longitude: 78)> Size: 28MB
[7107360 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 21kB 2015-01-31 ... 2015-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
Attributes: (12/31)
    GRIB_paramId:                             217016
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      2652
    GRIB_typeOfLevel:                         hybrid
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_pv:                                  [0.00000000e+00 2.00000000e+01 ...
    GRIB_shortName:                           c5h8
    GRIB_units:                               kg kg**-1
    long_name:                                Isoprene
    units:                                    kg kg**-1
    standard_name:                            mass_fraction_of_isoprene_in_air

In [3]:
isoprene.to_dataframe()

c5h8
valid_time          latitude longitude                                  
2015-01-31 00:00:00 49.25    -124.849999999999994  1.409947714137161e-10
                             -124.099999999999994  1.610374056326691e-10
                             -123.349999999999994  2.573719015686038e-10
                             -122.599999999999994  2.158815348707321e-10
                             -121.849999999999994  8.343814528188886e-11
...                                                                  ...
2015-12-31 21:00:00 24.50    -70.100000000000023   9.999994032438992e-26
                             -69.350000000000023   9.999994032438992e-26
                             -68.600000000000023   9.999994032438992e-26
                             -67.850000000000023   9.999994032438992e-26
                             -67.100000000000023   9.999994032438992e-26

[7107360 rows x 1 columns]

In [4]:
PATH_2=os.path.join('..','CAMS_79_variables_2015.pkl')
get_data = pd.read_pickle(PATH_2) ## load the data with all variables
get_data.head()

,NAME,fips,u10,v10,d2m,t2m,bcaod550,duaod550,lsm,msl,omaod550,pm1,pm2p5,pm10,ssaod550,suaod550,z,sp,aod469,aod550,aod670,aod865,aod1240,tcco,tc_c2h6,tchcho,tc_h2o2,tc_oh,tc_c5h8,tc_ch4,tc_hno3,tcno2,tc_no,gtco3,tc_pan,tc_c3h8,tcso2,tcwv,co,aermr04,aermr05,aermr06,c2h6,hcho,h2o2,aermr09,aermr07,aermr10,aermr08,oh,c5h8,hno3,no2,no,go3,pan,c3h8,aermr01,aermr02,aermr03,q,aermr11,so2,t,lai_hv,lai_lv,asn,sd,tcw,aermssdus,aermssdum,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
1807,Autauga,1001,-0.200146064977586,-0.063139664377364,285.978472435061292,292.38995334774603,0.006054659234753,0.014350361065572,0.986987539115746,101853.94994671997847,0.070681076555562,0.0000000099635,0.000000013514302,0.000000018288582,0.010253812351381,0.068377941023866,1312.642868868908636,100263.777544493044843,0.204441045481307,0.169717963223539,0.132901470823519,0.095844281326341,0.058096693160641,0.000853928004128,0.000003319909326,0.00000486102841,0.0000092597362,0.000000004873514,0.000005343348716,0.009963674431452,0.000004351392412,0.000002958334071,0.000000423381409,0.006295894795233,0.000009034799496,0.000000593398993,0.000002659670892,28.829037105964286,0.000000166880125,0.000000000965658,0.000000001695855,0.00000000053321,0.000000000535215,0.00000000494567,0.000000000961481,0.000000000181144,0.000000006415276,0.000000000199705,0.000000003463706,0.000000000000018,0.000000028902851,0.000000000696285,0.000000006453807,0.000000000359204,0.000000058068934,0.000000003498004,0.000000000222113,0.000000000027005,0.000000002175179,0.000000000259489,0.010814258340913,0.000000002218969,0.000000005053632,293.074929958592008,3.906738033237082,2.625992421689393,0.879570618778252,0.002644332590413,28.959441674085568,0.000002889655166,0.000005830597078,0.000003517081551,0.000000113408627,0.00000212017901,0.000000527903881,0.000011615072522,0.00000024110953,0.00001958907466,0.000001058182286,0.00000698515136,0.000001591096342
3114,Baldwin,1003,-0.514061111900449,-0.055965832921291,289.580801629997609,293.778017119677884,0.006530976735661,0.01784201872005,0.537505880455493,101801.418469400014146,0.069876511314192,0.000000008619804,0.000000012433803,0.000000017087712,0.014432011992982,0.068713181147813,223.036092952870092,101530.528758873566403,0.212318409886544,0.177394700896452,0.140476273969049,0.103609990883983,0.065323897724192,0.000833146032746,0.000003254820527,0.000003749051857,0.000010291086327,0.0000000049224,0.000001932543277,0.010090164004706,0.000004602461444,0.000002532872,0.000000382954343,0.00620988427069,0.000007659301211,0.000000545777525,0.000002016266076,31.792455034791761,0.000000144878986,0.000000000956535,0.000000001601613,0.000000000530348,0.000000000493586,0.000000002971965,0.000000000984635,0.000000000173049,0.000000005075683,0.000000000199796,0.00000000256079,0.000000000000031,0.000000009658897,0.000000000826927,0.000000004832641,0.000000000255672,0.000000062071373,0.00000000242375,0.000000000191789,0.000000000075559,0.00000000624186,0.000000003573332,0.012999198008351,0.000000001848929,0.000000002621937,294.581250467674522,3.085466070828208,1.942851221974209,0.879967104467305,0.000798281848789,31.889914096880265,0.000003557287148,0.000007259018678,0.000004609346305,0.000000124978711,0.000001869518171,0.000000567206143,0.000011922211371,0.000000338396213,0.000027616363374,0.00000270922079,0.000007371878646,0.000000933822224
0,Barbour,1005,-0.241356325051469,-0.077598989480741,286.538842095393761,293.219803052757698,0.006207189934389,0.014654921128112,0.989722920389318,101840.971726896241307,0.071408701833154,0.000000010425384,0.000000014072415,0.000000019072908,0.011283476539523,0.067394012162067,1082.55658972910669,100529.946296566282399,0.205624514573034,0.170948291308717,0.134189798298054,0.097195376879819,0.059393341608523,0.000849545513645,0.0000032949445,0.000004749621799,0.000009604165664,0.000000004870981,0.000004664072346,0.0099912355942,0.000004339077672,0.00

In [5]:
percentile_75 = get_data['c5h8'].quantile(0.90) ## calculate 90th percentile
print(percentile_75)
threshold=percentile_75

2.292370310687971e-08


In [6]:
SHAPE_PATH=os.path.join('..','..','..','..','Shapefiles','county_shapefiles','2015_county_shapefile','cb_2015_us_county_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,01,005,00161528,0500000US01005,01005,Barbour,06,2291820706,50864677,"POLYGON ((-85.74803 31.61918, -85.74544 31.618..."
1,01,023,00161537,0500000US01023,01023,Choctaw,06,2365954971,19059247,"POLYGON ((-88.47323 31.89386, -88.46888 31.930..."
2,01,035,00161543,0500000US01035,01035,Conecuh,06,2201896058,6643480,"POLYGON ((-87.42720 31.26436, -87.42551 31.268..."
3,01,051,00161551,0500000US01051,01051,Elmore,06,1601876535,99850740,"POLYGON ((-86.41333 32.75059, -86.37115 32.750..."
4,01,065,00161558,0500000US01065,01065,Hale,06,1667804583,32525874,"POLYGON ((-87.87046 32.76244, -87.86818 32.765..."
...,...,...,...,...,...,...,...,...,...,...
3228,45,019,01252740,0500000US45019,45019,Charleston,06,2372842394,1144346152,"MULTIPOLYGON (((-79.50795 33.02008, -79.50713 ..."
3229,45,077,01248015,0500000US45077,45077,Pickens,06,1285536060,40612589,"MULTIPOLYGON (((-82.86687 34.61742, -82.86451 ..."
3230,46,123,01265784,0500000US46123,46123,Tripp,06,4176233698,13272785,"POLYGON ((-100.23091 43.49989, -100.23044 43.5..."
3231,47,073,01639752,0500000US47073,47073,Hawkins,06,1261443215,32545400,"POLYGON ((-83.28890 36.37879, -83.28250 36.382..."


In [7]:
geometry_column=county_gdf['geometry']
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x                   y
0    0   -85.748031999999995  31.619180999999998
     1   -85.745435000000001  31.618897999999998
     2   -85.742650999999995  31.621258999999998
     3   -85.741739999999993  31.619402999999998
     4   -85.739812999999998  31.621810000000000
...                      ...                 ...
3232 29 -101.385870999999995  34.748376999999998
     30 -101.434904000000003  34.747419999999998
     31 -101.471562000000006  34.747461999999999
     32 -101.629256999999996  34.747648999999996
     33 -101.629396000000000  34.750056000000001

[1037334 rows x 2 columns]

In [8]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.
        And then to find the fraction of time above percentile
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            pm_above_df: Dataframe consisting the values of the variables interpolated in the county.
    '''   
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    extract_val=len(longitude) ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    latitude=lat_lon.loc[(var),'y']    ## get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= isoprene.interp(longitude=lon_list, latitude=lat_list)
    
## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=year_avg_finer.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)
    
    initial_df=pd.concat(get_vals)
    summary = initial_df.groupby("valid_time").mean(["latitude", "longitude"])
    df=summary

    total_rows=len(df)

    c=0
    for i in range(0, total_rows):
        if df['c5h8'].values[i] > threshold:
            c=c+1

## convert number to a fraction
    frac_time=(c/total_rows)*100
    pm_above_df=pd.DataFrame({'isoprene above percentile':[frac_time]})


    return pm_above_df

In [9]:
len(county_gdf)

3233

In [10]:
%%time

## get the values for all counties

df_list_below=[]

for i in range(0,len(county_gdf)): ## loop for the all list of counties
    try:
        df_list_below.append(single_county(i))
    except:
        pass

CPU times: total: 57min 29s
Wall time: 58min 13s


In [11]:
concatenate_df=pd.concat(df_list_below)
final_df=concatenate_df.reset_index(drop=True)
final_df

,isoprene above percentile
0,29.888059701492537
1,34.626865671641795
2,30.410447761194032
3,32.350746268656714
4,34.738805970149258
...,...
3228,16.305970149253731
3229,30.000000000000000
3230,0.000000000000000
3231,28.395522388059703


In [12]:
## lets first create a fips column in the original shape file

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','AFFGEOID','GEOID','LSAD','ALAND','AWATER'], axis=1)
county_gdf

,NAME,geometry,fips
0,Barbour,"POLYGON ((-85.74803 31.61918, -85.74544 31.618...",1005
1,Choctaw,"POLYGON ((-88.47323 31.89386, -88.46888 31.930...",1023
2,Conecuh,"POLYGON ((-87.42720 31.26436, -87.42551 31.268...",1035
3,Elmore,"POLYGON ((-86.41333 32.75059, -86.37115 32.750...",1051
4,Hale,"POLYGON ((-87.87046 32.76244, -87.86818 32.765...",1065
...,...,...,...
3228,Charleston,"MULTIPOLYGON (((-79.50795 33.02008, -79.50713 ...",45019
3229,Pickens,"MULTIPOLYGON (((-82.86687 34.61742, -82.86451 ...",45077
3230,Tripp,"POLYGON ((-100.23091 43.49989, -100.23044 43.5...",46123
3231,Hawkins,"POLYGON ((-83.28890 36.37879, -83.28250 36.382...",47073


In [13]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,isoprene above percentile
0,Barbour,"POLYGON ((-85.74803 31.61918, -85.74544 31.618...",1005,29.888059701492537
1,Choctaw,"POLYGON ((-88.47323 31.89386, -88.46888 31.930...",1023,34.626865671641795
2,Conecuh,"POLYGON ((-87.42720 31.26436, -87.42551 31.268...",1035,30.410447761194032
3,Elmore,"POLYGON ((-86.41333 32.75059, -86.37115 32.750...",1051,32.350746268656714
4,Hale,"POLYGON ((-87.87046 32.76244, -87.86818 32.765...",1065,34.738805970149258
...,...,...,...,...
3228,Charleston,"MULTIPOLYGON (((-79.50795 33.02008, -79.50713 ...",45019,16.305970149253731
3229,Pickens,"MULTIPOLYGON (((-82.86687 34.61742, -82.86451 ...",45077,30.000000000000000
3230,Tripp,"POLYGON ((-100.23091 43.49989, -100.23044 43.5...",46123,0.000000000000000
3231,Hawkins,"POLYGON ((-83.28890 36.37879, -83.28250 36.382...",47073,28.395522388059703


In [14]:
county_df=merged_df.dropna()
county_df.head()

,NAME,geometry,fips,isoprene above percentile
0,Barbour,"POLYGON ((-85.74803 31.61918, -85.74544 31.618...",1005,29.888059701492537
1,Choctaw,"POLYGON ((-88.47323 31.89386, -88.46888 31.930...",1023,34.626865671641795
2,Conecuh,"POLYGON ((-87.42720 31.26436, -87.42551 31.268...",1035,30.410447761194032
3,Elmore,"POLYGON ((-86.41333 32.75059, -86.37115 32.750...",1051,32.350746268656714
4,Hale,"POLYGON ((-87.87046 32.76244, -87.86818 32.765...",1065,34.738805970149258


In [15]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,isoprene above percentile
0,Barbour,1005,29.888059701492537
1,Choctaw,1023,34.626865671641795
2,Conecuh,1035,30.410447761194032
3,Elmore,1051,32.350746268656714
4,Hale,1065,34.738805970149258
...,...,...,...
3228,Charleston,45019,16.305970149253731
3229,Pickens,45077,30.000000000000000
3230,Tripp,46123,0.000000000000000
3231,Hawkins,47073,28.395522388059703


In [16]:
county_var.to_pickle('isoprene_above_percentile.pkl')